In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install "IPython[all]"

     |████████████████████████████████| 204kB 3.5MB/s 


In [0]:
import sys
sys.path.append('/content/drive/My Drive/mask_rcnn_segmentation/keras-mask-rcnn/Mask_RCNN')
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import visualize
import numpy as np
import argparse
import imutils
import random
import cv2
import os

Using TensorFlow backend.


# Change location of weights for Mask-RCNN trained on coco dataset and coco labels .txt file

In [0]:

args = {'weights':'/content/drive/My Drive/mask_rcnn_segmentation/keras-mask-rcnn/mask_rcnn_coco.h5',
       'labels':'/content/drive/My Drive/mask_rcnn_segmentation/keras-mask-rcnn/coco_labels.txt',
       }

# Loading the model

In [0]:

# load the class label names from disk, one label per line
CLASS_NAMES = open(args["labels"]).read().strip().split("\n")
random.seed(42)


class SimpleConfig(Config):
	# give the configuration a recognizable name
	NAME = "coco_inference"

	# set the number of GPUs to use along with the number of images
	# per GPU
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

	# number of classes (we would normally add +1 for the background
	# but the background class is *already* included in the class
	# names)
	NUM_CLASSES = len(CLASS_NAMES)

# initialize the inference configuration
config = SimpleConfig()

# initialize the Mask R-CNN model for inference and then load the
# weights
print("[INFO] loading Mask R-CNN model...")
model = modellib.MaskRCNN(mode="inference", config=config,model_dir=os.getcwd())
model.load_weights(args["weights"], by_name=True)

W0903 17:03:08.592751 140054834157440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0903 17:03:08.649856 140054834157440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0903 17:03:08.666399 140054834157440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0903 17:03:08.718776 140054834157440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2139: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0903 17:03:08.722592 140054834

[INFO] loading Mask R-CNN model...


W0903 17:03:11.487931 140054834157440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2239: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.

W0903 17:03:12.033555 140054834157440 deprecation_wrapper.py:119] From /content/drive/My Drive/mask_rcnn_segmentation/keras-mask-rcnn/Mask_RCNN/mrcnn/model.py:341: The name tf.log is deprecated. Please use tf.math.log instead.

W0903 17:03:12.045387 140054834157440 deprecation.py:323] From /content/drive/My Drive/mask_rcnn_segmentation/keras-mask-rcnn/Mask_RCNN/mrcnn/model.py:399: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0903 17:03:12.053485 140054834157440 deprecation.py:506] From /content/drive/My Drive/mask_rcnn_segmentation/keras-m

# Predict masks and remove background on task images

In [0]:
# path to task images directory
dir_ = '/content/drive/My Drive/mask_rcnn_segmentation/keras-mask-rcnn/images/'
for im_name in os.listdir(dir_):
  image = cv2.imread(dir_+str(im_name))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  print("[INFO] making predictions with Mask R-CNN...")
  r = model.detect([image], verbose=1)[0]
  print(r["rois"].shape[0])
  for i in range(0, r["rois"].shape[0]):
    classID = r["class_ids"][i]
    # continue if detected class is not person
    if CLASS_NAMES[classID]!='person':
      continue
    mask = r["masks"][:, :, i]
    mask = mask.astype('uint8')
    print(mask.shape)
    seg_image = np.zeros(shape =image.shape)
    seg_image[:,:,0] = np.multiply(image[:,:,0], mask)
    seg_image[:,:,1] = np.multiply(image[:,:,1], mask)
    seg_image[:,:,2] = np.multiply(image[:,:,2], mask)
    mask[mask==1] = 255
    
    cv2.imwrite('/content/drive/My Drive/mask_rcnn_segmentation/keras-mask-rcnn/task_masks/'+str(im_name)+'_mask.png', mask)
    
    cv2.imwrite('/content/drive/My Drive/mask_rcnn_segmentation/keras-mask-rcnn/results/'+str(im_name)+'_seg.png', seg_image)